In [ ]:
%%capture --no-stderr
%pip install -U langgraph langchain-community tavily-python pandas groq langchain-groq langgraph

In [ ]:
%pip install -qU duckduckgo-search langchain-community

In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("GROQ_API_KEY")

GROQ_API_KEY: ··········


In [ ]:
import getpass
import os
import re
import numpy as np
import requests
import random
from datetime import date, datetime
from typing import Annotated, Optional
from typing_extensions import TypedDict

from sentence_transformers import SentenceTransformer
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnableConfig, RunnableLambda, RunnableBranch
from langchain_groq import ChatGroq
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph.message import AnyMessage, add_messages


class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [ ]:
import re
import numpy as np
import requests
from sentence_transformers import SentenceTransformer
from langchain_core.tools import tool

# Load FAQ text from local storage
with open("/content/company_faq.txt", "r", encoding="utf-8") as file:
    faq_text = file.read()[:1000]

# Split documents
docs = [{"page_content": txt} for txt in re.split(r"(?=\n##)", faq_text)]

# Initialize Hugging Face Embedding Model
hf_model = SentenceTransformer("all-MiniLM-L6-v2")


class VectorStoreRetriever:
    def __init__(self, docs: list, vectors: np.ndarray):
        self._arr = np.array(vectors)
        self._docs = docs

    @classmethod
    def from_docs(cls, docs):
        embeddings = hf_model.encode(
            [doc["page_content"] for doc in docs], convert_to_numpy=True
        )
        return cls(docs, embeddings)

    def query(self, query: str, k: int = 5) -> list[dict]:
        query_embedding = hf_model.encode([query], convert_to_numpy=True)[0]
        scores = query_embedding @ self._arr.T
        top_k_idx = np.argpartition(scores, -k)[-k:]
        top_k_idx_sorted = top_k_idx[np.argsort(-scores[top_k_idx])]
        return [
            {**self._docs[idx], "similarity": scores[idx]} for idx in top_k_idx_sorted
        ]


# Create retriever with Hugging Face embeddings
retriever = VectorStoreRetriever.from_docs(docs)


@tool
def lookup_policy(query: str) -> str:
    """Consult the company policies to check whether certain options are permitted.
    Use this before making any flight changes performing other 'write' events."""
    docs = retriever.query(query, k=1)
    return "\n\n".join([doc["page_content"] for doc in docs])

import random

@tool
def welcome_msg(query: str) -> str:
    """Returns a funny welcome message based on the query input."""
    funny_welcome_messages = [
        "Welcome! I was expecting you... No, really, I was! 😎",
        "Oh hey, another human! Welcome aboard! 🤖✨",
        "Welcome! Hope you brought snacks, because we're about to have a great chat! 🍕",
        "Greetings! If you need wisdom, humor, or just a virtual high five, I'm here! ✋😆",
        "Hey there! I'm like Google, but with more personality and fewer ads. 😜",
    ]
    return random.choice(funny_welcome_messages)


@tool
def check_tool_usage(tool_name: str) -> str:
    """Check if a specific tool has been called in the model's execution."""
    return f"Checking if the tool '{tool_name}' is being used... If you're seeing this, then yes! 🚀"



In [ ]:
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda

from langgraph.prebuilt import ToolNode


def handle_tool_error(state) -> dict:
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls
    return {
        "messages": [
            ToolMessage(
                content=f"Error: {repr(error)}\n please fix your mistakes.",
                tool_call_id=tc["id"],
            )
            for tc in tool_calls
        ]
    }


def create_tool_node_with_fallback(tools: list) -> dict:
    return ToolNode(tools).with_fallbacks(
        [RunnableLambda(handle_tool_error)], exception_key="error"
    )


def _print_event(event: dict, _printed: set, max_length=1500):
    current_state = event.get("dialog_state")
    if current_state:
        print("Currently in: ", current_state[-1])
    message = event.get("messages")
    if message:
        if isinstance(message, list):
            message = message[-1]
        if message.id not in _printed:
            msg_repr = message.pretty_repr(html=True)
            if len(msg_repr) > max_length:
                msg_repr = msg_repr[:max_length] + " ... (truncated)"
            print(msg_repr)
            _printed.add(message.id)

In [ ]:
from langchain_groq import ChatGroq
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnableConfig
# from langchain_ollama import ChatOllama
from datetime import date, datetime

class Assistant:
    def __init__(self, runnable: Runnable):
        self.runnable = runnable

    def __call__(self, state: State, config: RunnableConfig):
        while True:
            configuration = config.get("configurable", {})
            passenger_id = configuration.get("User info", None)
            state = {**state, "user_info": passenger_id}
            result = self.runnable.invoke(state)
            # If the LLM happens to return an empty response, we will re-prompt it
            # for an actual response.
            if not result.tool_calls and (
                not result.content
                or isinstance(result.content, list)
                and not result.content[0].get("text")
            ):
                messages = state["messages"] + [("user", "Respond with a real output.")]
                state = {**state, "messages": messages}
            else:
                break
        return {"messages": result}


# Haiku is faster and cheaper, but less accurate
# llm = ChatAnthropic(model="claude-3-haiku-20240307")
# local_llm = "llama3.2:3b-instruct-fp16"
# llm = ChatOllama(model=local_llm, temperature=1)
local_llm = "llama3-8b-8192"
llm =ChatGroq(model=local_llm, temperature=0.5)
# You could swap LLMs, though you will likely want to update the prompts when
# doing so!
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-4-turbo-preview")

primary_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful customer support assistant for GOutam Sachdev Company to using tools . "
            " Use the provided tools to for welcome , policies and check tools answer according to toools return values . "
            " When searching, be persistent. Expand your query bounds if the first search returns no results. "
            " If a search comes up empty, expand your search before giving up."
            "\n\nCurrent user:\n<User>\n user_id 1 \n</User>"
            "\nCurrent time: {time}.",
        ),
        ("placeholder", "{messages}"),
    ]
).partial(time=datetime.now)

part_1_tools = [
    DuckDuckGoSearchRun(),
    lookup_policy,
    check_tool_usage,
    welcome_msg

]
part_1_assistant_runnable = primary_assistant_prompt | llm.bind_tools(part_1_tools)

config = {
    "configurable": {
        # The passenger_id is used in our flight tools to
        # fetch the user's flight information
        "user_id": 1,
        # Checkpoints are accessed by thread_id
        "thread_id": 123,
    }
}
res=part_1_assistant_runnable.invoke(
    {"messages": [("user", "Will I receive a confirmation after canceling? ")]}, config

    )
print(res.content)

In [ ]:
print(res)

content='' additional_kwargs={'tool_calls': [{'id': 'call_p67e', 'function': {'arguments': '{"query":"Will I receive a confirmation after canceling?"}', 'name': 'welcome_msg'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 1371, 'total_tokens': 1444, 'completion_time': 0.060833333, 'prompt_time': 0.207789702, 'queue_time': 0.534690983, 'total_time': 0.268623035}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-24dc917d-f195-462d-acdb-7c5f878a2a19-0' tool_calls=[{'name': 'welcome_msg', 'args': {'query': 'Will I receive a confirmation after canceling?'}, 'id': 'call_p67e', 'type': 'tool_call'}] usage_metadata={'input_tokens': 1371, 'output_tokens': 73, 'total_tokens': 1444}


In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain_groq import ChatGroq
from langchain_core.tools import Tool
from datetime import datetime

# Define a system prompt to provide context to the agent.
system_prompt = (
    "You are a helpful customer support assistant for BItswits Company using tools. "
    "Use the provided tools to handle welcome messages, policy lookups, and tool usage checks. "
    "When searching, be persistent. Expand your query bounds if the first search returns no results. "
    "If a search comes up empty, expand your search before giving up. "
    f"Current time: {datetime.now()}."
)

# Define your tools as Tool objects.
qa_tool = Tool(
    name="lookup_policy",
    func=lookup_policy,
    description="Consults company policies to check if certain options are permitted."
)

welcome_tool = Tool(
    name="welcome_msg",
    func=welcome_msg,
    description="Returns a humorous welcome message."
)

tool_usage_tool = Tool(
    name="check_tool_usage",
    func=check_tool_usage,
    description="Checks if a specific tool has been called during the model's execution."
)

# Group all tools together.
all_tools = [qa_tool, welcome_tool, tool_usage_tool]

# Initialize your LLM; here we use ChatGroq with your preferred model.
local_llm = "llama-3.3-70b-versatile"
llm = ChatGroq(model=local_llm, temperature=0.5)

# Initialize the agent using ZERO_SHOT_REACT_DESCRIPTION.
# We pass the system prompt as the "prefix" in agent_kwargs.
agent = initialize_agent(
    tools=all_tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_kwargs={"prefix": system_prompt},
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate'

)

# Run the agent with a sample query.
query = "if i cancel my ticket will i have to panelty"
response = agent.run(query)
print(response)




> Entering new AgentExecutor chain...
To answer the question about canceling a ticket and potential penalties, I need to consult the company policies.

Action: lookup_policy
Action Input: 'cancellation penalty'
Observation: 
Q: Can I update someone else's schedule?
A: No, you can only update schedules linked to your user ID.

Q: What if I enter an invalid date or time?
A: The system will reject invalid formats and notify you to use the correct date (YYYY-MM-DD) and time (HH:MM).

Q: Can I change the schedule to a past date?
A: No, all updated schedules must be for today or a future date.

3. Canceling a Schedule
Q: How do I cancel an appointment?
A: You can cancel an appointment by providing the schedule ID. The system will verify ownership before removing the schedule.

Q: Can I cancel an appointment for another user?
A: No, only the user who created the appointment can cancel it.

Q: Will I receive a confirmation after canceling?
A: Yes, the system will confirm that the appointment

In [ ]:
from langchain_core.tools import Tool

qa_tool = Tool(
    name="lookup_policy",
    func=lookup_policy,
    description="Consults company policies to determine whether certain options are permitted."
)

welcome_tool = Tool(
    name="welcome_msg",
    func=welcome_msg,
    description="Returns a humorous welcome message for the user."
)

tool_usage_tool = Tool(
    name="check_tool_usage",
    func=check_tool_usage,
    description="Checks if a specific tool has been called during the model's execution."
)

# Group all tools into a single variable for later use
all_tools = [qa_tool, welcome_tool, tool_usage_tool]


Input data type: <class 'dict'>
Input data: {'messages': [HumanMessage(content='Can I cancel an appointment for another user?', additional_kwargs={}, response_metadata={})]}
An error occurred: If 'exception_key' is specified then input must be a dictionary.However found a type of <class 'list'> for input


TypeError: 'AIMessage' object is not subscriptable